---
# Notebook para automatizar fiscalização
---

Esse notebook tem por propósito automatizar a fiscalização por monitoramento.

## 1. Configuração Inicial

### 1.1. Configurações Iniciais

#### 1.1.1. Importação de bibliotecas

In [2]:
import pandas as pd 
from datetime import datetime as dt 
from dateutil.relativedelta import relativedelta
from connection import DadosEscriturais, DadosOperacoes, DadosCadastrais, DadosParametrizados, DadosDebitos
from external_connection import DadosCadastraisRFB, DadosCadastraisJUCERR
from EFD.EFDLocal import EFDLocal
import os 
import warnings

relatorio = []


#### 1.1.2. Configurações básicas

In [3]:
data_inicio=dt(2020, 1, 1)
data_fim=dt(2020, 12, 31)
cnpj = '18283407000210'
cgf = '24.027688-2'

valor_minimo_significancia = 100000

endereco_pasta_audit = r'C:\Users\Jimmy_Usuário\Documents\Auditor_Fiscal\Planos de Trabalho - DIFIS\022024\Fiscalizações\OS 22762024 - 34797837000206\Dados'

endereco_efd = r'C:\Users\Jimmy_Usuário\Documents\Auditor_Fiscal\Planos de Trabalho - DIFIS\022024\Fiscalizações\OS 22762024 - 34797837000206\Dados\EFDs'



##### 1.1.2.1. Tratamentos adicionais

In [4]:
num_months = (data_fim.year - data_inicio.year) * 12 + data_fim.month - data_inicio.month + 1   # Calcular quantidade de meses de fiscalização
meses_audit = [(data_inicio + relativedelta(months=x)).strftime('%m/%Y') for x in range(num_months)] # Transformar quantidade de meses de fiscalização em formato MM/YYYY

warnings.filterwarnings('ignore')   # Ignorar alguns avisos de deprecação de funções do pandas

### 1.2. Coleta de EFD e GIM

In [5]:
# Coleta de GIM
gim = DadosEscriturais.GIM(cgf=cgf, data_inicio=data_inicio, data_fim=data_fim)
obrigatoriedade = DadosEscriturais.BuscarObrigatoriedade(cgf=cgf, data_inicio=data_inicio, data_fim=data_fim)

# Salvar GIM importada (para documentação de auditoria)
gim.save(path=endereco_pasta_audit+r'\escrituracao.xlsx')


Banco de dados já existe!


In [ ]:
# Coleta de EFD
dados_efd = EFDLocal(cnpj=cnpj, mes_ref=meses_audit, local=endereco_efd)

### 1.3. Coleta de documentos adicionais

In [6]:
parametrizacoes = DadosParametrizados

### 1.4. Coleta da Inscrição no CNPJ

In [7]:
dados_cnpj = DadosCadastraisRFB(cnpj=cnpj)
dados_jucerr = DadosCadastraisJUCERR(cnpj=cnpj)


### 1.5. Coleta da inscrição no CGF

In [8]:
cadastro = DadosCadastrais.CadastroAtualEstadual(cnpj)
cadastro_historico = DadosCadastrais.HistoricoCadastroEstadual(cnpj)

cadastro.save(path=endereco_pasta_audit+r'\cadastro_atual_estadual.xlsx')
cadastro_historico.save(path=endereco_pasta_audit+r'\historico_cadastro_estadual.xlsx')

### 1.6. Coleta de Dsot e Pagamentos Antecipados

In [9]:
dsot = DadosDebitos.DSOT(cgf, data_inicio, data_fim)
pag_antecipado = DadosDebitos.DebitosAntecipacaoParcialPagos(cgf, data_inicio, data_fim)

dsot.save(path=endereco_pasta_audit+r'\dsot_periodo.xlsx')
pag_antecipado.save(path=endereco_pasta_audit+r'\pag_antecipado_periodo.xlsx')

### 1.7. Coleta de Documentos Fiscais Não Escriturais

In [10]:

NFeEntrada = DadosOperacoes.NotasFiscaisEntrada(cnpj=cnpj, data_inicio=data_inicio, data_fim=data_fim)
NFeSaida = DadosOperacoes.NotasFiscaisSaida(cnpj=cnpj, data_inicio=data_inicio, data_fim=data_fim)
NFCe = DadosOperacoes.NotasFiscaisConsumidor(cnpj=cnpj, data_inicio=data_inicio, data_fim=data_fim)

notas = pd.ExcelWriter(path=endereco_pasta_audit+'\documentos_fiscais.xlsx')
NFeEntrada.to_excel(notas, sheet_name='NotasEntrada', index=False)
NFeSaida.to_excel(notas, sheet_name='NotaSaida', index=False)
NFCe.to_excel(notas, sheet_name='NFCe', index=False)
notas.close()


#### 1.2.1. Obtendo notas fiscais escrituradas extemporaneamente

A obrigação da escrituração se baseia na data de entrada, enquanto que a busca de notas fiscais no banco de dados nesse notebook se refere à data de emissão. 
Além disso, há a possibilidade de escrituração de nota fiscal extemporânea. 
Assim, deve-se obter as notas fiscais de emissão fora do período de abrangência da fiscalização. 

In [11]:
# Extrair C100
C100 = dados_efd.C100

# Obter chaves de acesso a serem importadas
notas_entradas_a_importar = C100[((C100['COD_SIT'].isin(['1', '7'])) | (C100['DT_DOC'] < data_inicio)) & (C100['IND_OPER']=='0') & (C100['COD_MOD']=='55')]['CHV_NFE'].values
notas_saidas_a_importar = C100[((C100['COD_SIT'].isin(['1', '7'])) | (C100['DT_DOC'] < data_inicio)) & (C100['IND_OPER']=='1') & (C100['COD_MOD']=='55')]['CHV_NFE'].values
notas_consumidor_a_importar = C100[((C100['COD_SIT'].isin(['1', '7'])) | (C100['DT_DOC'] < data_inicio)) & (C100['IND_OPER']=='1') & (C100['COD_MOD']=='65')]['CHV_NFE'].values


if notas_entradas_a_importar.size > 0:
    print(f'Adicionando {notas_entradas_a_importar.size} documentos fiscais extemporâneos - Entrada.')
    NFeEntrada.AdicionarNotasFiscaisExtemporaneas(tuple(notas_entradas_a_importar))

if notas_saidas_a_importar.size > 0:
    print(f'Adicionando {notas_saidas_a_importar.size} documentos fiscais extemporâneos - Saída.')
    NFeSaida.AdicionarNotasFiscaisExtemporaneas(tuple(notas_entradas_a_importar))

if notas_consumidor_a_importar.size > 0:
    print(f'Adicionando {notas_consumidor_a_importar.size} documentos fiscais extemporâneos - Consumidor.')
    NFCe.AdicionarNotasFiscaisConsumidorExtemporaneas(tuple(notas_entradas_a_importar))

Importando 12 documentos fiscais extemporâneos - Entrada.


## 2. Auditoria de Obrigações Acessórias

### 2.1. Verificação da inscrição no CNPJ

- Verificar se está inscrito;
- Verificar o período de inscrição;


In [12]:
temp = dados_cnpj.BuscarCadastro()

if temp['SituacaoCadastral'].values in ('Ativa', 'Ativo'):
    relatorio.append(f'Contribuinte está inscrito no CNPJ desde {pd.to_datetime(temp["DataCadastro"].values[0]).strftime("%d/%m/%Y")}.')

else:
    relatorio.append(f'Contribuinte está com inscrição desativada! {temp["SituacaoCadastral"].values}.')

del temp

### 2.2. Verificação da inscrição no CGF 

- Verificar status da inscrição estadual no período de abrangência
- Verificar regime de apuração
- Verificar se o sócio-administrador está batendo com banco estadual

In [13]:
temp = cadastro_historico.retrieve()
if temp['SituacaoCadastral'].values[-1] in ('A', 'N'):
    relatorio.append(f'Contribuinte está ativo no CGF.')

else:
    relatorio.append(f'Contribuinte está com inscrição pendente! {temp["SituacaoCadastral"].values[-1]}.')

del temp 

In [14]:
# raise NotImplementedError('Ainda não está pronto! Falta importar Socios do CGF!')
socios_jucerr = dados_jucerr.BuscarSocios()
# socios_cgf = DadosCadastrais.BuscarSocios(cnpj)

### 2.3. Verificação de Emissão de Documentos Fiscais


In [15]:
if NFeEntrada.quantidade()['quantidade'] == 0:
    relatorio.append('O contribuinte NÃO está recebendo notas fiscais (de entrada) no período de abrangência.')
    print("Foi encontrada uma irregularidade!")
else:
    relatorio.append('O contribuinte está recebendo notas fiscais (de entrada) no período de abrangência.')

if NFeSaida.quantidade()['quantidade'] == 0 and NFCe.quantidade()['quantidade'] == 0:
    relatorio.append('O contribuinte NÃO está emitindo nota fiscal de saída (inclusive NFCe)!')
    print("Foi encontrada uma irregularidade!")
else:
    relatorio.append('O contribuinte está emitindo Nota Fiscal Eletrônica de saída / Nota Fiscal ao Consumiddor Eletrônica!')


### 2.4. Verificação da declaração da Guia de Informação Mensal do ICMS

In [16]:
# raise NotImplementedError('Excluir meses em que contribuinte estava pelo Simples Nacional')
data_atual = data_inicio
lista_periodos_sem_gim = []
obrigatoriedade_gim = [data.strftime('%m/%Y') for data in obrigatoriedade[obrigatoriedade['OB_TIPO']=='GIM']['OB_DATA']]


gim_importada = gim.retrieve()
while data_atual <= data_fim:
    if data_atual.strftime('%m/%Y') not in gim_importada["PERÍODO"].values and data_atual.strftime('%m/%Y') in obrigatoriedade_gim:
        lista_periodos_sem_gim.append(data_atual.strftime('%m/%Y'))
    data_atual += relativedelta(months=1)

if len(lista_periodos_sem_gim) != 0:
    relatorio.append('Há meses com omissão de declaração de Guia de Informações Mensais no período de abrangência.')
else:
    relatorio.append('O contribuinte transmitiu todas as Guia de Informações Mensais no período de abrangência.')

del gim_importada

### 2.5. Verificação da declaração da Escrituração Fiscal Digital 

In [ ]:
data_atual = data_inicio
lista_periodos_sem_efd = []
obrigatoriedade_efd = [data.strftime('%m/%Y') for data in obrigatoriedade[obrigatoriedade['OB_TIPO']=='EFD']['OB_DATA']]

INFO0000 = dados_efd.INFO0000
meses_escriturados = [x.strftime('%m/%Y') for x in INFO0000['DT_INI']]

while data_atual <= data_fim:
    if data_atual.strftime('%m/%Y') not in meses_escriturados and data_atual.strftime('%m/%Y') in obrigatoriedade_efd:
        lista_periodos_sem_efd.append(data_atual.strftime('%m/%Y'))
    data_atual += relativedelta(months=1)

if len(lista_periodos_sem_efd) != 0:
    relatorio.append('Há meses com omissão de declaração de Escrituração Fiscal Digital no período de abrangência.')
else:
    relatorio.append('O contribuinte transmitiu todas as Escrituração Fiscal Digital no período de abrangência.')
    
del INFO0000 
del meses_escriturados

#### 2.5.1. Verificação de declaração da Escrituração Fiscal Digital com Inventário (Bloco H)

Verificar se há declarado estoque início e final (meses de dezembro, janeiro e fevereiro)

Tem que gerar relatório.

Verificar se validador aceita bloco H em outros meses.

In [5]:
def extrair_anos(data_inicio, data_fim):
    return list(range(data_inicio.year, data_fim.year + 1))

In [6]:
# Obtendo dados de declaração de H (estoque)
H001 = dados_efd.H001 

# Filtrando por blocos H declarados
H001 = H001[H001['IND_MOV']=='0']


# Filtrando por EFD de dezembro a fevereiro
INFO0000 = dados_efd.INFO0000 
INFO0000 = INFO0000[INFO0000['DT_INI'].dt.month.isin([12, 1, 2])]   # Precisa ser declarado o bloco H de dezembro a fevereiro

# Verificando datas de declaração
INFO0000 = INFO0000[INFO0000['ID_0000'].isin(H001['ID_0000'])]

anos_com_H = INFO0000['DT_INI'].dt.year
exercicios_fiscalizados = extrair_anos(data_inicio, data_fim)



# Se quantidade de bloco H < qtde anos, informar ausência de bloco H
if len(anos_com_H) < len(exercicios_fiscalizados):
    exercicios_com_omissao_H = [x for x in exercicios_fiscalizados if x not in anos_com_H]
    relatorio.append(f'Há omissão de declaração do bloco H (Estoque) da Escrituração Fiscal Digital nos exercícios de: {exercicios_com_omissao_H}')
else:
    relatorio.append('Não foram constatadas omissões de bloco H nas Escriturações Fiscais Digitais no período de abrangência.')

del INFO0000 
del H001
del anos_com_H 
del exercicios_fiscalizados

## 3. Auditoria dos Dados Cadastrais

### 3.1. Verificação do regime de apuração de ICMS

In [ ]:
regimes = cadastro_historico.retrieve()[['DataAlteracao', 'DataAlteracaoAnterior', 'OptanteSimples', 'OptanteSimplesAnterior']]
regimes = regimes[(regimes['DataAlteracao']>=data_inicio) | (regimes['DataAlteracao'].isin(regimes[regimes['DataAlteracao']>=data_inicio]['DataAlteracaoAnterior'].values))]
regimes.columns = ['Periodo Final', 'Periodo Inicial', 'RegimeSimples', 'RegimeSimplesPeriodo']
regimes

In [17]:
data_atual = data_inicio
regimes_sn = []

if 1 in regimes['RegimeSimplesPeriodo']:
    regimes_simples = regimes[regimes['RegimeSimplesPeriodo']==0]
    for idx, periodo in regimes_simples.iterrows():
        periodo_inicial_regime = periodo['Periodo Inicial'].strftime('%d/%m/%Y')
        periodo_final_regime = periodo['Periodo Final'].strftime('%d/%m/%Y')
        regimes_sn.append([periodo_inicial_regime, periodo_final_regime])

if regimes['RegimeSimples'].values[-1] != 0:
    periodo_inicial_regime = regimes['Periodo Inicial'].values[-1].strftime('%d/%m/%Y')
    periodo_final_regime = regimes['Periodo Final'].values[-1].strftime('%d/%m/%Y')
    regimes_sn.append([periodo_inicial_regime, periodo_final_regime])

if len(regimes_sn) != 0:
    relatorio.append('Há meses com Opção de Recolhimento pelo Simples Nacional no período de abrangência.')
else:
    relatorio.append('O contribuinte recolheu pelo regime normal durante todo o período de abrangência.')
    
del regimes

### 3.2. Verificação de cadastro de atividades econômicas

## 4. Auditoria da Situação Fiscal

### 4.1. Verificação da existência de débitos constantes do DSOT

Os débitos precisam estar inseridos no âmbito da fiscalização (OS)

In [18]:
dsot_existente = dsot.retrieve()
dsot_ativo = dsot_existente[dsot_existente['Suspensao']=='N']
if dsot_existente.empty:
    relatorio.append('Não há débitos registrados nos sistemas da SEFAZ-RR.')
elif dsot_ativo.empty: 
    relatorio.append(f'Há apenas débitos suspensos registrados nos sistemas da SEFAZ-RR')
else: 
    valor_total_debito = sum(dsot_ativo['ValorPrincipal'] + dsot_ativo['ValorMulta'] + dsot_ativo['ValorJuros'] + dsot_ativo['ValorCorrecao'])
    relatorio.append(f'Há débitos registrados nos sistemas da SEFAZ-RR no montante de{valor_total_debito}')

del dsot_ativo
del dsot_existente

## 5. Auditoria das Operações e Prestações de Entrada

In [ ]:
# Trazer dados para memória RAM 
nfe_entrada = NFeEntrada.retrieve()
eventos_desemb = DadosOperacoes.EventosDesembaraco(tuple(nfe_entrada[nfe_entrada['DESTINO']=='INTERESTADUAL']['ChaveAcesso'].values)).retrieve()
eventos = DadosOperacoes.EventosNotas(tuple(nfe_entrada['ChaveAcesso'].values)).retrieve()

### 5.1. Verificação de notas fiscais de entrada não escrituradas

Regras:
- Escriturado: não verifica
- Não escriturado e desembaraçado (interestadual): Exige baseado na data do desembaraço
- Não escriturado e não desembaraçado (interestadual): não exige qualquer conformidade
- Não escriturado e não desembaraçado (interno) e com evento de não reconhecimento: não exige qualquer conformidade
- Não escriturado e não desembaraçado (interno) e sem evento de não reconhecimento: exige escrituração baseado no mês da data de emissão

In [6]:
# Primeira parte - Obtendo documentos de entrada não escriturados
C100 = dados_efd.C100


nfe_n_escriturada = nfe_entrada[~nfe_entrada['ChaveAcesso'].isin(C100.CHV_NFE)]

# Segunda parte - Obtendo documentos desembaraçados se for interestadual
nfe_n_escriturada = nfe_n_escriturada.merge(eventos_desemb, left_on='ChaveAcesso', right_on='CHV_ACC', how='left')
nfe_n_escriturada = nfe_n_escriturada[(nfe_n_escriturada['ChaveAcesso'].isin(eventos_desemb['CHV_ACC'])) | (nfe_n_escriturada['DESTINO'] != 'INTERESTADUAL')]

# Terceira parte - Filtrando pelos documentos com eventos
eventos_canc = eventos[eventos['CodigoEvento'].isin(['210220', '210240'])]
nfe_n_escriturada = nfe_n_escriturada[~(nfe_n_escriturada['ChaveAcesso'].isin(eventos_canc['ChaveAcesso']))]

# Quarta parte - Verificar data de escrituração devida
def calculate_data_escrituracao(linha):
    if linha['DATA_PASSAGEM'] is None:
        return linha['DATA_PASSAGEM']
    elif str(linha['CFOP.ITEM'])[0] in ['2', '5']:
        return linha['DT.EMISSÃO'] + relativedelta(days=2)
    else:
        return linha['DT.EMISSÃO']

nfe_n_escriturada['DataEscrituraçãoDevida'] = nfe_n_escriturada.apply(calculate_data_escrituracao, axis=1)


# Quinta parte - Indicar existência de notas não escrituradas 
if not nfe_n_escriturada.empty: 
    relatorio.append(f'Existem {len(nfe_n_escriturada)} Notas Fiscais válidas e não escrituradas no período.')

else:
    relatorio.append(f'Não existem Notas Fiscais não escrituradas no período.')

del nfe_n_escriturada 
del eventos_canc 


### 5.2. Verificação de notas fiscais não desembaraçadas 

Criam-se duas tabelas:
- Tabela de Nota não escriturada e não desembaraçada;
- Tabela de nota escriturada e não desembaraçada.

Regras:
- Data de Desembaraço: Data de emissão + 90 (caso não escriturado) OU data de escrituração (DT_E_S).
- Remove notas com passagem no posto e com código de evento de cancelamento.

In [20]:
# Primeira parte - Obtendo documentos interestaduais
nfe_n_desemb = nfe_entrada[(nfe_entrada['DESTINO']=='INTERESTADUAL')&(nfe_entrada['FINALIDADE'].isin(['NORMAL', 'DEVOLUÇÃO', 'N/A']))]

# Segunda parte - Obtendo eventos de desembaraço para filtrar por não desembaraço
nfe_n_desemb = nfe_n_desemb[~nfe_n_desemb['ChaveAcesso'].isin(eventos_desemb['CHV_ACC'])]


# Terceira parte - Verificando quais foram escrituradas
nfe_n_desemb = nfe_n_desemb.merge(C100[['CHV_NFE', 'DT_E_S']], left_on='ChaveAcesso', right_on='CHV_NFE', how='left')
nfe_n_desemb['Escriturado'] = ~nfe_n_desemb['CHV_NFE'].isna() 


# Quarta Parte Encontrando data de desembaraço devida
nfe_n_desemb['DataDesembaraçoDevida'] = [linha['DT.EMISSÃO'] + relativedelta(days=90) if not linha['Escriturado'] else linha['DT_E_S'] for idx, linha in nfe_n_desemb.iterrows() ]

# Quinta Parte - Filtrando pelos documentos com eventos
eventos_canc = eventos[eventos['CodigoEvento'].isin(['210220', '210240'])]
nfe_n_desemb = nfe_n_desemb[~(nfe_n_desemb['ChaveAcesso'].isin(eventos_canc['ChaveAcesso']))]

# Sexta parte - Indicar existência de notas não desembaraçadas
if not nfe_n_desemb.empty: 
    print('Foi encontrado Notas Fiscais válidas e não desembaraçadas no período')
    relatorio.append(f'Existem {len(nfe_n_desemb)} Notas Fiscais válidas e não desembaraçadas no período.')

else:
    relatorio.append(f'Não existem Notas Fiscais não desembaraçadas no período.')

del nfe_n_desemb 
del eventos_canc

### 5.3. Verificação de notas fiscais canceladas escrituradas na entrada

Verificam-se as notas fiscais que não deveriam ter sido escrituradas por terem eventos de cancelamento. 


|Código|Descrição|
|---|---|
|00|Documento Regular|
|01|Escrituração extemporânea de documento regular|
|02|Documento cancelado|
|03|Escrituração extemporânea de documento cancelado|
|04|Doc denegado|
|05|Numeração inutilizada|
|06|DF-e Complementar|
|07|Escrituração extemporânea de DF-e complementar|
|08|DF-e emitido com base em Regime Especial ou Norma Específica|

In [ ]:
# Primeira parte - Obter notas fiscais escrituradas
C100 = dados_efd.C100

# Segunda parte - Filtrar por Notas de entrada e sem COD_SIT de cancelados 
C100 =  C100[(C100['IND_OPER']=='0')]
C100_canc = C100[(~C100['COD_SIT'].isin(['00', '01', '06', '07', '08', '0', '1', '6', '7', '8']))]
C100_n_canc = C100[(C100['COD_SIT'].isin(['00', '01', '06', '07', '08', '0', '1', '6', '7', '8']))]

# Terceira parte - Obter últimos eventos dos documentos escriturados e renomear colunas
eventos = eventos.sort_values(by='SequenciaEvento', ascending=False).drop_duplicates(subset = 'ChaveAcesso', keep='first')
eventos.rename(columns={'ChaveAcesso': 'CHV_NFE'}, inplace=True)

# Quarta parte - Filtrando por eventos de cancelamentos
eventos_canc = eventos[eventos['CodigoEvento'].isin(['110111', '210220', '210240'])]

# Quinta parte - Filtrar por notas escrituradas e com eventos de cancelamento
nota_canc_escriturada = C100_n_canc.merge(eventos_canc, on = 'CHV_NFE', how='inner', validate='one_to_one') 

# Relatando problemas
if not nota_canc_escriturada.empty:
    print('Foram encontradas Notas Fiscais inválidas e escrituradas no período.')
    relatorio.append(f'Existem {len(nota_canc_escriturada)} Notas Fiscais inválidas e escrituradas no período.')
else:
    print('Não foram encontradas Notas Fiscais inválidas e escrituradas no período.')

# Sexta parte - Filtrar por notas escrituradas como canceladas e sem eventos de cancelamento
nota_n_canc_escriturada_canc = C100_n_canc.merge(eventos, on = 'CHV_NFE', how='left_only', validate='one_to_one') 

# Relatando problemas
if not nota_n_canc_escriturada_canc.empty:
    print('Foram encontradas Notas Fiscais sem eventos de cancelamento e escrituradas como canceladas no período.')
    relatorio.append(f'Existem {len(nota_n_canc_escriturada_canc)} Notas Fiscais válidas e escrituradas como canceladas no período.')
else:
    print('Não foram encontradas Notas Fiscais inválidas e escrituradas no período.')

# Retirando dados da memória RAM
del nota_canc_escriturada
del nota_n_canc_escriturada_canc
del eventos_canc
del C100 
del C100_canc 
del C100_n_canc

### 5.4. Verificação de Escrituração Duplicada no C100 em entrada

Verifica se há dupla escrituração de uma mesma nota em períodos distintos.

Regra:
- Verificar se há chave de acesso dobrada
- Verificar se há o conjunto "série, número e remetente" duplicados.

In [ ]:
# Extraindo os dados
C100 = dados_efd.C100
C100 =  C100[(C100['IND_OPER']=='0')]    # Filtrando pelas entradas

# Enriquecendo com dados de mês de referência de EFD
INFO0000 = dados_efd.INFO0000[['ID_0000', 'DT_INI']]
INFO0000['DT_INI'] = INFO0000['DT_INI'].strftime('%m/%Y')
INFO0000.columns = ['ID_0000', 'MesReferencia']
C100 = INFO0000.merge(C100, on = 'ID_0000', how='inner', validate = 'one_to_many')

# Verificando chaves dobradas
chaves_dobradas = C100[C100.duplicated(subset='CHV_NFE')]['CHF_NFE'].unique() 
chaves_dobradas['Irregularidade'] = 'Chave de Acesso Duplicada'

# Verificando conjunto "serie, numero e remetente" duplicados
notas_dobradas = C100[C100.duplicated(subset=['COD_PART', 'COD_MOD', 'SER', 'NUM_DOC'])]
notas_dobradas['Irregularidade'] = 'Conjunto de Código Participante, Série, Número de Nota e Modelo de Documento Duplicados'

escrituracao_duplicada = pd.concat([chaves_dobradas, notas_dobradas], axis=0)

# Relatando problemas
if not escrituracao_duplicada.empty:
    print('Foram encontradas Notas Fiscais de entrada duplicadas no período de abrangência da fiscalização')
    relatorio.append(f'Existem {len(escrituracao_duplicada)} Notas Fiscais de entrada duplicadas no período de abrangência de fiscalização.')
else:
    print('Não foram encontradas Notas Fiscais de entrada duplicadas no período de abrangência da fiscalização')

# Retirando dados da memória RAM
del C100 
del INFO0000
del chaves_dobradas 
del notas_dobradas 
del escrituracao_duplicada

### 5.4. Verificação de Escrituração do C170 (Obrigação Acessória)

Este procedimento serve para identificar inconsistências entre o C170 e as Notas Fiscais. 

<i>A inconsistência é gerada quando o contribuinte não respeita o NUM_ITEM (que deve ser igual entre o C170 e a nota respectiva)</i>

In [ ]:
# Preparando o 0200
INFO0200 = dados_efd.INFO0200 

INFO0200.drop_duplicates(subset=['COD_ITEM', 'DESCR_ITEM', 'COD_NCM', 'COD_BARRA'], inplace=True)

if INFO0200['COD_ITEM'].duplicated().sum() > 0:
    inconsistencia_0200 = INFO0200[INFO0200['COD_ITEM'].duplicated()]
    relatorio.append('Foi encontrada inconsistência no registro 0200 com mesmo COD_ITEM de descrições distintas.')

### 5.5. Verificação de créditos de ICMS

#### 5.3.1. C170 x 0200 x NF-e

Pontos verificados:
- Alíquotas 
- Direitos à crédito por CFOP


Regras:
- Verificar se há direito a crédito de cada produto;
  - Se o produtor for tributado, considerar com direito a crédito
  - Se o produto não for tributado, exigir modificação no C170 para não ter direito a crédito
  - Se o produto for sujeito a ST, exigir modificação no C170 para não ter direito a crédito


##### 5.3.1.1. Parametrizar o registro 0200

In [ ]:
raise NotImplementedError('Optar pelo modelo do Lucas Bem')
# Preparando o 0200
INFO0200 = dados_efd.INFO0200
INFO0200.drop_duplicates(subset=['COD_ITEM', 'DESCR_ITEM', 'COD_BARRA', 'COD_NCM'], inplace=True)


if INFO0200['COD_ITEM'].duplicated().sum() > 0:
    inconsistencia_0200 = INFO0200[INFO0200['COD_ITEM'].duplicated()]
    print('Foi encontrada inconsistência no registro 0200 com mesmo COD_ITEM de descrições distintas e/ou NCM e GTIN distintos.')
    relatorio.append('Foi encontrada inconsistência no registro 0200 com mesmo COD_ITEM de descrições distintas e/ou NCM e GTIN distintos.')

# Continuando a análise com apenas o primeiro registro do COD_ITEM
INFO0200.drop_duplicates(subset='COD_ITEM', inplace=True)

# Adicionando tabela de GTIN 
GTIN = parametrizacoes.BuscarGtin()
INFO0200 = INFO0200.merge(GTIN, left_on = 'COD_BARRA', right_on='GTIN', how='left', validate='many_to_one')

# Adicionando tabela de NCM 
NCM = parametrizacoes.BuscarNcmParametrizados()
INFO0200 = INFO0200.merge(NCM, left_on = 'COD_NCM', right_on='NCM', how='left', validate='many_to_one')

# Parametrizando Alíquota    - Depende da data e da UF de remessa, logo ficará a cargo de parametrizar com as notas
for idx, linha in INFO0200.iterrows():
    if pd.notna(linha['CLASSE']): # Se houver parametrização por GTIN
        parametrizacao = linha['CLASSE']
    elif pd.notna(linha['PARAMETRIZAÇÃO']):  # Se houver parametrização por NCM
        parametrizacao = linha['PARAMETRIZAÇÃO']
    else: 
        parametrizacao = '4202'     # Salvo contrário, considerar tributado.
    INFO0200.loc[idx, 'CODIGO_PARAMETRIZADO'] = parametrizacao

INFO0200.drop_duplicates(subset='COD_ITEM', inplace=True)

INFO0200.head()

##### 5.3.1.2. Inserir Parametrização no C170

In [17]:
# Primeira parte - Obtendo documentos de entrada não escriturados
C170 = dados_efd.C170
C100 = dados_efd.C100
C170_completo = C100[['ID_C100', 'CHV_NFE', 'DT_E_S']].merge(C170, on = ['ID_C100'], validate='one_to_many', how='right') # Filtrando apenas por documentos com C170

# Segunda Parte - Inserindo APENAS código de parametrização e descrição do produto
C170_completo = C170_completo.merge(INFO0200[['COD_ITEM', 'DESCR_ITEM', 'CODIGO_PARAMETRIZADO']], on='COD_ITEM', how='left', validate='many_to_one')


# Terceira Parte - Acrescentar parametrização de CFOP
CFOP = parametrizacoes.BuscarCFOPTributados()
C170_completo = C170_completo.merge(CFOP, on='CFOP', how='left', validate='many_to_one')

# Quarta parte - Reduzindo a tabela de C170 modificado
C170_completo = C170_completo[['CHV_NFE', 'NUM_ITEM', 'DT_E_S', 'COD_ITEM', 'QTD', 'UNID', 'DESCR_ITEM', 'VL_ITEM', 
                         'VL_DESC', 'CFOP', 'VL_BC_ICMS', 'ALIQ_ICMS', 'VL_ICMS', 
                         'CODIGO_PARAMETRIZADO', 'GERA_CREDITO']]

# Quinta parte - Padronizando nome de colunas 
C170_completo.columns = ['CHV_NFE', 'NUM_ITEM', 'DT_E_S_C170', 'COD_ITEM_C170', 'QTD_C170', 'UNID_C170', 'DESCR_ITEM_C170', 'VL_ITEM_C170', 
                         'VL_DESC_C170', 'CFOP_C170', 'VL_BC_ICMS_C170', 'ALIQ_ICMS_C170', 'VL_ICMS_C170', 
                         'CODIGO_PARAMETRIZADO_C170', 'GERA_CREDITO_C170']

##### 5.3.1.3. Inserir parametrização nas NF-e de Entrada

In [24]:
# Obtendo apenas campos necessários
NFe_ENTRADA = nfe_entrada[['ChaveAcesso', 'Nº.ITEM', 'DT.EMISSÃO', 'DESC.ITEM', 'CFOP.ITEM', 'CLASSE.FRONT', 'GTIN', 'NCM', 'BC.ICMS.ITEM', 'ALIQ', 'ICMS.ITEM', 'ICMS.DESON.ITEM']]
NFe_ENTRADA = NFe_ENTRADA.copy()

# Convert columns to string safely
NFe_ENTRADA.loc[:, 'CFOP.ITEM'] = NFe_ENTRADA['CFOP.ITEM'].astype(str)
NFe_ENTRADA.loc[:, 'ChaveAcesso'] = NFe_ENTRADA['ChaveAcesso'].astype(str)
NFe_ENTRADA.loc[:, 'Nº.ITEM'] = NFe_ENTRADA['Nº.ITEM'].astype(str)

# Adicionando a parametrização
NFe_ENTRADA = NFe_ENTRADA.merge(CFOP[['CFOP', 'GERA_CREDITO']], left_on='CFOP.ITEM', right_on='CFOP', how='left', validate='many_to_one')

# Mudando nome de colunas para ficar padronizado
NFe_ENTRADA.drop(columns='CFOP', inplace=True)
NFe_ENTRADA.columns = ['CHV_NFE', 'NUM_ITEM', 'DT_EMISSÃO_NFe', 'DESC.ITEM_NFe', 'CFOP.ITEM_NFe', 'CLASSE.FRONT_NFe',
                        'GTIN_NFe', 'NCM_NFe', 'BC.ICMS.ITEM_NFe', 'ALIQ_NFe', 'ICMS.ITEM_NFe', 'ICMS.DESON.ITEM_NFe',
                        'GERA_CREDITO_NFe']

##### 5.3.1.4. Verificação de creditamento de ICMS de notas fiscais com CFOP que não dão direito ao crédito

- Direito a crédito via NFe x Direito a Crédito por C170 
    
    Verifica-se se há casos em que haja aproveitamento indevido de créditos no C170 quando a NF-e não permitir ter o direito (CFOP que não dão direito ao crédito).


In [ ]:
# Cruzando C170 modificado com NF-e de entrada
sem_direito_a_cred = C170_completo.merge(NFe_ENTRADA, on=['CHV_NFE', 'NUM_ITEM'], how='left', suffixes=['_C170', '_NFe'])

# Verificando direitos a crédito
sem_direito_a_cred = sem_direito_a_cred[(sem_direito_a_cred['GERA_CREDITO_NFe']==False) & (sem_direito_a_cred['GERA_CREDITO_C170']==True) & (sem_direito_a_cred['VL_ICMS_C170']>0)]

# Verificando se há alguma irregularidade
if not sem_direito_a_cred.empty and sem_direito_a_cred['VL_ICMS_C170'].sum() >= valor_minimo_significancia:
    resultado = f'Foi encontrado indício de irregularidade de direitos de créditos no C170 - Por Item - Quantidade: {len(sem_direito_a_cred)}'
    relatorio.append([resultado])
    print(resultado)
else:
    print('Não foi encontrado discrepância significativa no direito ao creditamento no C170.')

##### 5.3.1.5. Verificação de alíquota do C170 (comparando com NF-e)

- Alíquota via NFe x Alíquota C170

    Primeiramente, verifica-se se a alíquota do C170 é diferente da alíquota da NF-e, por item.
    Não podemos fazer pelo valor de produto ou valor de ICMS pelo fato de o contribuinte acabar escriturando de forma incorreta (foi constatado que ele preenche "NUM_ITEM" diferentes do da nota fiscal em si).
    Caso o valor da alíquota seja diferente, gerar relatório para verificar manualmente a irregularidade.

    <i>Observação: notas emitidas por empresas do Simples Nacional naturalmente serão apresentadas nesse relatório. Por isso não há outro procedimento de auditoria para buscar notas emitidas por SN e com creditamento de ICMS</i>

In [ ]:
raise NotImplementedError('Parei aqui na reunião com Papito')
# Cruzar C170 modificado com NF-e de entrada
cruzamento_efd_nfe_item = C170_completo.merge(NFe_ENTRADA, on=['CHV_NFE', 'NUM_ITEM'], how='left', suffixes=['_C170', '_NFe'])

# Verificar diferença de alíquotas
cruzamento_efd_nfe_item['Diferença_Alíquotas'] = cruzamento_efd_nfe_item['ALIQ_ICMS_C170']>cruzamento_efd_nfe_item['ALIQ_NFe']

# Filtrar por diferenças significativas
cruzamento_efd_nfe_item = cruzamento_efd_nfe_item[cruzamento_efd_nfe_item['Diferença_Alíquotas']==True]

# Relatar
if not cruzamento_efd_nfe_item.empty and cruzamento_efd_nfe_item['VL_ICMS_C170'].sum() >= valor_minimo_significancia:
    resultado = f'Foi encontrado indício de irregularidade de alíquotas no C170 - Por Item - Quantidade: {len(cruzamento_efd_nfe_item)}'
    relatorio.append([resultado])
    print(resultado)
else:
    print('Não foi encontrado discrepância significativa.')

#### 5.3.2. C190 x NF-e (itens adicionados)

Pontos verificados:
- Valor total de produto (VL_OPR)
- Valor total de Base de Cálculo
- Valor total de ICMS 

##### 5.3.2.1. Preparar C190

In [28]:
# Preparar C190
C190 = dados_efd.C190 
C100 = dados_efd.C100 
C190_completo = C100[C100['IND_OPER'=='0']][['ID_C100', 'CHV_NFE', 'DT_E_S', 'IND_OPER']].merge(C190, on='ID_C100', how='left', validate='one_to_many') # Filtrando apenas por notas de entrada

# Agrupar por CFOP
C190_mod = C190_completo.groupby(['CHV_NFE', 'CST_ICMS', 'ALIQ_ICMS'], as_index=False)[['VL_OPR', 'VL_BC_ICMS', 'VL_ICMS']].sum()

# Padronizar nome de colunas 
C190_mod.columns = ['CHV_NFE', 'CST_ICMS', 'ALIQ_ICMS', 'VL_OPR_C190', 'VL_BC_ICMS_C190', 'VL_ICMS_C190']

##### 5.3.2.2. Preparar NFe por CST, Chave de Acesso e Aliq

In [29]:
# Preparar NFe por ChaveAcesso/CFOP/CST/ALIQ 
nfe_cst = nfe_entrada[['ChaveAcesso', 'CST',  'ALIQ', 'VLR.LIQ.ITEM', 'BC.ICMS.ITEM','ICMS.ITEM']]

# Completar com 0 nos campos vazios (ALIQ, ICMS.ITEM, BC.ICMS.ITEM, DESCON.ITEM)
nfe_cst.fillna(0, inplace=True)

# Agrupar por CHV_NFE, CFOP, CST, ALIQ
nfe_cst = nfe_cst.groupby(['ChaveAcesso', 'CST', 'ALIQ'], as_index=False)[['VLR.LIQ.ITEM', 'BC.ICMS.ITEM','ICMS.ITEM']].sum()

# Alterar nome de colunas para padronizar 
nfe_cst.columns = ['CHV_NFE', 'CST_ICMS', 'ALIQ_ICMS', 'VL_OPR_NFE', 'VL_BC_ICMS_NFE', 'VL_ICMS_NFE']

# Tratando dados
nfe_cst['CST_ICMS'] = nfe_cst['CST_ICMS'].astype(int).astype(str)
nfe_cst['CHV_NFE'] = nfe_cst['CHV_NFE'].astype(str)

# Filtrando pelas NF-es que foram escrituradas (Ver próximo comentário para entender essa decisão)
nfe_cst = nfe_cst[nfe_cst['CHV_NFE'].isin(C190_mod['CHV_NFE'])]

<i>Explicação do filtro por NF-e escriturada:

Existem 3 possibilidades de encontrar linhas da tabela nfe_cst sem correspondente C190:
- Alíquota escriturada de forma incorreta (ex.: 17% no C190 e 12% na NFe)
- CST escriturado de forma incorreta (ex: CST 40 - isento - no C190 e CST 00 - tributado - na NFe)
- Chave de acesso inexistente (ex.: nota fiscal não escriturada OU nota fiscal escriturada extemporaneamente)

Como o objetivo desse cruzamento é verificar se o C190 está com aproveitamento de crédito adequado, e considerando que a não escrituração já gerou inconsistências no cruzamento de escrituração (item 5.1), decidiu-se por filtrar as NF-es apenas pelas escrituradas. 

Assim, o resultado será limitado aos dois primeiros pontos (alíquota ou CST inconsistentes, ou nota fiscal extemporânea)

##### 5.3.2.4. Tratar equivalência de CST

Conforme tabela CST, alguns CST são equivalentes:

- CST 10 - Tributada e com cobrança do ICMS por ST (na nota) 

Equivale a:

- CST 60 - ICMS cobrado anteriormente por ST (na escrituração do C190)

In [38]:
nfe_cst.loc[nfe_cst[nfe_cst['CST_ICMS']=='10'].index, 'CST_ICMS'] = '60'
nfe_cst.loc[nfe_cst[nfe_cst['CST_ICMS']=='60'].index, 'ALIQ_ICMS'] = 0.0 # Se for ST, considerar alíquota de nota igual a 0


# Agrupar novamente por CHV_NFE, CFOP, CST, ALIQ
nfe_cst = nfe_cst.groupby(['CHV_NFE', 'CST_ICMS', 'ALIQ_ICMS'], as_index=False)[['VL_OPR_NFE', 'VL_BC_ICMS_NFE', 'VL_ICMS_NFE']].sum()


##### 5.3.2.3. Verificação sintética de créditos de ICMS

In [ ]:
# Unir C190 com NFe por CHV_NFE, CST e ALIQ
inconsistencias_c190 = C190_mod.merge(nfe_cst, on=['CHV_NFE', 'CST_ICMS', 'ALIQ_ICMS'], how='outer', indicator=True, validate='one_to_one')

# Filtrar por inconsistencias (linha do C190 que não tenha linha NFe e vice-versa)
inconsistencias_c190 = inconsistencias_c190[inconsistencias_c190['_merge']!='both']

# Relatar
if not inconsistencias_c190.empty and inconsistencias_c190['VL_ICMS_C190'].fillna(0).sum() >= valor_minimo_significancia:
    resultado = f'Foi encontrado indício de irregularidade de aproveitamento de crédito - Por Nota/CST - Quantidade: {len(inconsistencias_c190)}'
    relatorio.append([resultado])
    print(resultado)
else:
    print('Não foi encontrada nenhuma discrepância significativa.')

#### 5.3.3. Verificação de creditamento de ICMS - Transporte de CT-es que não dão direito ao crédito

In [ ]:
raise NotImplementedError("Ainda não estipulado view para obter isso")

#### 5.3.4. Verificação do valor recolhido a título de Antecipação Parcial com valor escriturado em Outros Créditos


Para verificar os valores de ajuste a crédito, somam-se:
- Valores pagos a título de Antecipação Parcial 
- Valores a título de crédito presumido 
- Valores de Crédito de Ativo Imobilizado 


In [ ]:
# Obtendo valores pagos a título de antecipação parcial
antecip_parcial = debitos.BuscarDebitosAntecipacaoParcialPagos(cgf=cgf, data_inicio=data_inicio, data_fim=data_fim)

In [ ]:
antecip_parcial

In [ ]:
E110 = pd.DataFrame() 

for mes_ref in dados_efd.keys():
    E110 = pd.concat([E110, pd.DataFrame(dados_efd[mes_ref].RetornarCampo('E110'))])
E110

In [ ]:
C197 = pd.DataFrame() 

for mes_ref in dados_efd.keys():
    C197 = pd.concat([C197, pd.DataFrame(dados_efd[mes_ref].RetornarCampo('C197'))])
C197

In [ ]:
E111 = pd.DataFrame() 

for mes_ref in dados_efd.keys():
    E111 = pd.concat([E111, pd.DataFrame(dados_efd[mes_ref].RetornarCampo('E111'))])
E111

In [ ]:
# Filtrar por notas de remessa por remetentes do Simples Nacional
nfe_entrada_sn = nfe_entrada[nfe_entrada['REGIME']=='SIMPLES']
nfe_entrada_sn

In [ ]:
nfe_entrada_sn[['ALIQ', 'ICMS.ITEM', 'ICMS.DESON.ITEM']]

In [ ]:
nfe_entrada_sn.columns

In [ ]:
# Primeira parte - Obtendo documentos de entrada não escriturados
E110 = pd.DataFrame() 

for mes_ref in dados_efd.keys():
    E110 = pd.concat([E110, pd.DataFrame(dados_efd[mes_ref].RetornarCampo('E110'))])
E110